# GWChemPlots: Prepare the data in CHS format for GWChemPlot
La CHS mantiene una red de control de la calidad de las aguas subterráneas. Los datos se pueden descargar en https://www.chsegura.es/es/cuenca/redes-de-control/calidad-en-aguas-subterraneas/acceso-a-los-datos/

La tabla que facilita la CHS tiene una estructura del tipo: estación, fecha, parámetro, otros datos y parámetros. Esta estructura se cambia a otra del tipo:
estacion, fecha, iones mayoritarios, otros parámetros, que utiliza AquaChem para hacer sus gráficos.

A partir de la estructura original ee graban 2 ficheros con la estructura modificada:

1. estacion, fecha, iones mayoritarios expresados en mg/L y en meq/L, otros parámetros y el CBE (Charge Balance Error)
1. Contenidos específicos utilizados por el módulo WQChartPy (https://github.com/jyangfsu/WQChartPy), del que se toman algunas funciones y se empaquetan en el módulo GWChemPlot.


In [1]:
# Mientras escribo pued cambiar algo en AquaChem.py
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import pandas as pd

import sys
path = r'C:\Users\solis\Documents\DEV\python3\CalidadAgua\GWChemPlot'
if path not in sys.path:
    sys.path.append(path)
from GWChemPlots import cbe

Leo el fichero descargado de la Web de la CHS

In [4]:
# vistazo al fichero descargado
fchs = r'E:\LSGB\data2db\chs_calidad\sinclinal_calasparra.csv'
gwa = pd.read_csv(fchs, dtype={'Cod Masa':'string', 'Cod. Parámetro':'string', 'UH Geo':'string'}, keep_default_na=False)

print(gwa.info())
gwa.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1132 entries, 0 to 1131
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Estación        1132 non-null   object 
 1   Fecha Toma      1132 non-null   object 
 2   Cod. Parámetro  1132 non-null   string 
 3   Nom. Parámetro  1132 non-null   object 
 4   Unidades        1132 non-null   object 
 5   Valor numérico  1132 non-null   float64
 6   Valor texto     1132 non-null   object 
 7   Coord. X        1132 non-null   int64  
 8   Coord. Y        1132 non-null   int64  
 9   Cod Masa        1132 non-null   string 
 10  Nombre Masa     1132 non-null   object 
 11  Municipio       1132 non-null   object 
 12  Provincia       1132 non-null   object 
 13  UH Geo          1132 non-null   string 
 14  UH Geo Nombre   1132 non-null   object 
 15  Acuifero        1132 non-null   object 
 16  Profundidad     1132 non-null   float64
 17  Nombre C.A      1132 non-null   o

,Estación,Fecha Toma,Cod. Parámetro,Nom. Parámetro,Unidades,Valor numérico,Valor texto,Coord. X,Coord. Y,Cod Masa,Nombre Masa,Municipio,Provincia,UH Geo,UH Geo Nombre,Acuifero,Profundidad,Nombre C.A
0,CA0708002,06/10/1981,CA,CALCIO,mg/L Ca,96.2,"96,2",633207,4243995,070.022,SINCLINAL DE CALASPARRA,Cieza,MURCIA,07.08,SINCLINAL DE CALASPARRA,SINCLINAL DE CALASPARRA,432.0,"Murcia, Región de"
1,CA0708002,06/10/1981,MG,MAGNESIO,mg/L Mg,75.4,"75,4",633207,4243995,070.022,SINCLINAL DE CALASPARRA,Cieza,MURCIA,07.08,SINCLINAL DE CALASPARRA,SINCLINAL DE CALASPARRA,432.0,"Murcia, Región de"
2,CA0708002,06/10/1981,RESIDUO,Residuo seco,mg/l,1440.0,1440,633207,4243995,070.022,SINCLINAL DE CALASPARRA,Cieza,MURCIA,07.08,SINCLINAL DE CALASPARRA,SINCLINAL DE CALASPARRA,432.0,"Murcia, Región de"
3,CA0708002,29/05/1992,CA,CALCIO,mg/L Ca,42.0,42,633207,4243995,070.022,SINCLINAL DE CALASPARRA,Cieza,MURCIA,07.08,SINCLINAL DE CALASPARRA,SINCLINAL DE CALASPARRA,432.0,"Murcia, Región de"
4,CA0708002,29/05/1992,MG,MAGNESIO,mg/L Mg,72.0,72,633207,4243995,070.022,SINCLINAL DE CALASPARRA,Cieza,MURCIA,07.08,SINCLINAL DE CALASPARRA,SINCLINAL DE CALASPARRA,432.0,"Murcia, Región de"


In [5]:
# simplifico nombre de columnas
new_col_names = {'Estación':'id', 'Fecha Toma':'fecha', 'Cod. Parámetro':'param', 'Nom. Parámetro':'param_name',
                 'Unidades' : 'uds' ,'Valor numérico':'v', 'Valor texto':'vstr', 'Coord. X': 'x', 'Coord. Y': 'y', 
                 'Cod Masa':'masub', 'Nombre Masa':'masub_name', 'Municipio': 'tm', 'Provincia':'prov',
                 'UH Geo':'uh','UH Geo Nombre':'uh_name', 'Acuifero':'acu', 'Profundidad':'prof', 'Nombre C.A': 'ca' }
gwa.rename(columns = new_col_names, inplace = True)

In [6]:
# ajusto los tipos de las columnas
gwa["id"] = gwa["id"].astype('string')
gwa["fecha"] = pd.to_datetime(gwa["fecha"], format='%d/%m/%Y')
gwa["param"] = gwa["param"].astype('string')
gwa["param_name"] = gwa["param_name"].astype('string')
gwa["uds"] = gwa["uds"].astype('string')
gwa["v"] = gwa["v"].astype('float32')
gwa["vstr"] = gwa["vstr"].astype('string')
gwa["x"] = gwa["x"].astype('float32')
gwa["y"] = gwa["y"].astype('float32')
gwa["masub"] = gwa["masub"].astype('category')
gwa["masub_name"] = gwa["masub_name"].astype('string')
gwa["tm"] = gwa["tm"].astype('string')
gwa["prov"] = gwa["prov"].astype('string')
gwa["uh"] = gwa["uh"].astype('category')
gwa["uh_name"] = gwa["uh_name"].astype('string')
gwa["acu"] = gwa["acu"].astype('category')
gwa["prof"] = gwa["prof"].astype('float32')
gwa["ca"] = gwa["ca"].astype('string')

In [7]:
# rango de fechas de mis datos
print("Minimum date:", gwa['fecha'].min())
print("Maximum date:", gwa['fecha'].max())

Minimum date: 1981-10-06 00:00:00
Maximum date: 2022-05-23 00:00:00


In [8]:
# Puntos de muestreo, rangos de fechas y número de datos
gwa.groupby('id')['fecha'].agg(['min', 'max', 'count'])

,min,max,count
id,,,
CA0708002,1981-10-06,2022-05-23,1132


In [9]:
# paso el contenido de param a minúsculas
gwa['param'] = gwa['param'].str.lower() 
gwa

,id,fecha,param,param_name,uds,v,vstr,x,y,masub,masub_name,tm,prov,uh,uh_name,acu,prof,ca
0,CA0708002,1981-10-06,ca,CALCIO,mg/L Ca,96.199997,"96,2",633207.0,4243995.0,070.022,SINCLINAL DE CALASPARRA,Cieza,MURCIA,07.08,SINCLINAL DE CALASPARRA,SINCLINAL DE CALASPARRA,432.0,"Murcia, Región de"
1,CA0708002,1981-10-06,mg,MAGNESIO,mg/L Mg,75.400002,"75,4",633207.0,4243995.0,070.022,SINCLINAL DE CALASPARRA,Cieza,MURCIA,07.08,SINCLINAL DE CALASPARRA,SINCLINAL DE CALASPARRA,432.0,"Murcia, Región de"
2,CA0708002,1981-10-06,residuo,Residuo seco,mg/l,1440.000000,1440,633207.0,4243995.0,070.022,SINCLINAL DE CALASPARRA,Cieza,MURCIA,07.08,SINCLINAL DE CALASPARRA,SINCLINAL DE CALASPARRA,432.0,"Murcia, Región de"
3,CA0708002,1992-05-29,ca,CALCIO,mg/L Ca,42.000000,42,633207.0,4243995.0,070.022,SINCLINAL DE CALASPARRA,Cieza,MURCIA,07.08,SINCLINAL DE CALASPARRA,SINCLINAL DE CALASPARRA,432.0,"Murcia, Región de"
4,CA0708002,1992-05-29,mg,MAGNESIO,mg/L Mg,72.000000,72,633207.0,4243995.0,070.022,SINCLINAL DE CALASPARRA,Cieza,MURCIA,07.08,SINCLINAL DE CALASPARRA,SINCLINAL DE CALASPARRA,432.0,"Murcia, Región de"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1127,CA0708002,2021-06-08,suo4,Sulfatos,mg/L SO4,225.000000,225,633207.0,4243995.0,070.022,SINCLINAL DE CALASPARRA,Cieza,MURCIA,07.08,SINCLINAL DE CALASPARRA,SINCLINAL DE CALASPARRA,432.0,"Murcia, Región de"
1128,CA0708002,2022-05-23,nh4,Amonio Total,mg/L NH4,1.300000,"1,3",633207.0,4243995.0,070.022,SINCLINAL DE CALASPARRA,Cieza,MURCIA,07.08,SINCLINAL DE CALASPARRA,SINCLINAL DE CALASPARRA,432.0,"Murcia, Región de"
1129,CA0708002,2022-05-23,ntotal,Nitrógeno total,mg/L N,1.200000,"1,2",633207.0,4243995.0,070.022,SINCLINAL DE CALASPARRA,Cieza,MURCIA,07.08,SINCLINAL DE CALASPARRA,SINCLINAL DE CALASPARRA,432.0,"Murcia, Región de"
1130,CA0708002,2022-05-23,phsitu,pH de campo (medida in situ),udpH,8.230000,"8,23 (26.2ºC)",633207.0,4243995.0,070.022,SINCLINAL DE CALASPARRA,Cieza,MURCIA,07.08,SINCLINAL DE CALASPARRA,SINCLINAL DE CALASPARRA,432.0,"Murcia, Región de"


In [10]:
# Inspecciono param_name y param
sorted((gwa['param_name'] + ' | ' + gwa['param']).unique())

['(m,p-xileno) (mezcla técnica) | xilm+p',
 '1,1,1,2-TETRACLOROETANO | 4cleta',
 '1,1,1-Tricloroetano (metilcloroformo) | tce',
 '1,1,2,2-TETRACLOROETANO | 4cleta2',
 '1,1,2-Tricloroetano | 3cleta',
 '1,1-DICLOROETANO | 2cleta',
 '1,1-DICLOROETENO | 2clete',
 '1,1-DICLOROPROPENO | 2clprope',
 '1,2,3-TRICLOROPROPANO | 3clpro',
 '1,2,3-Triclorobenceno | tcb123',
 '1,2,4-Triclorobenceno | tcb124',
 '1,2,4-trimetilbenceno | tmb1',
 '1,2-DIBROMO-3-CLOROPROPANO | 2br2clpro',
 '1,2-DIBROMOETANO | 12dibr',
 '1,2-DICLOROETENO | 2clete2',
 '1,2-DICLOROPROPANO | 2clpropa',
 '1,2-diclorobenceno (o-diclorobenceno) | o-dicloro',
 '1,2-dicloroetano (EDC, cloruro de etileno) | dce',
 '1,3,5-TRIMETILBENCENO | tmb2',
 '1,3,5-Triclorobenceno | tcb135',
 '1,3-DICLOROPROPANO | 2clpro',
 '1,3-diclorobenceno (m-diclorobenceno) | dcb-m',
 '1,4-diclorobenceno (p-diclorobenceno) | dcb-p',
 '2,2-DICLOROPROPANO | 2clpropa2',
 '2-CLOROTOLUENO | cltol',
 '4-CLOROTOLUENO | clto2',
 '4-ISOPROPILTOLUENO | isopropilto'

In [11]:
"""creo una nueva dataframe solo con mayoritarios y otros parámetros que me interesan"""

mayor = gwa[gwa['param'].isin(['na', 'k', 'ca', 'mg', 'cl', 'suo4', 'bicarb', 'co3', 'no3',
                             'cond 20º', 'cond20situ', 'cond25', 'phsitu'])]
mayor[0:5]

,id,fecha,param,param_name,uds,v,vstr,x,y,masub,masub_name,tm,prov,uh,uh_name,acu,prof,ca
0,CA0708002,1981-10-06,ca,CALCIO,mg/L Ca,96.199997,"96,2",633207.0,4243995.0,070.022,SINCLINAL DE CALASPARRA,Cieza,MURCIA,07.08,SINCLINAL DE CALASPARRA,SINCLINAL DE CALASPARRA,432.0,"Murcia, Región de"
1,CA0708002,1981-10-06,mg,MAGNESIO,mg/L Mg,75.400002,"75,4",633207.0,4243995.0,070.022,SINCLINAL DE CALASPARRA,Cieza,MURCIA,07.08,SINCLINAL DE CALASPARRA,SINCLINAL DE CALASPARRA,432.0,"Murcia, Región de"
3,CA0708002,1992-05-29,ca,CALCIO,mg/L Ca,42.000000,42,633207.0,4243995.0,070.022,SINCLINAL DE CALASPARRA,Cieza,MURCIA,07.08,SINCLINAL DE CALASPARRA,SINCLINAL DE CALASPARRA,432.0,"Murcia, Región de"
4,CA0708002,1992-05-29,mg,MAGNESIO,mg/L Mg,72.000000,72,633207.0,4243995.0,070.022,SINCLINAL DE CALASPARRA,Cieza,MURCIA,07.08,SINCLINAL DE CALASPARRA,SINCLINAL DE CALASPARRA,432.0,"Murcia, Región de"
7,CA0708002,1992-09-10,ca,CALCIO,mg/L Ca,67.000000,67,633207.0,4243995.0,070.022,SINCLINAL DE CALASPARRA,Cieza,MURCIA,07.08,SINCLINAL DE CALASPARRA,SINCLINAL DE CALASPARRA,432.0,"Murcia, Región de"


In [12]:
# Renombro algunos parámeros
mayor.loc[mayor['param'] == 'suo4', 'param'] = 'so4'
mayor.loc[mayor['param'] == 'bicarb', 'param'] = 'hco3'
mayor.loc[mayor['param'] == 'carb', 'param'] = 'co3'
mayor.loc[mayor['param'] == 'cond 20º', 'param'] = 'cond20'

In [13]:
# pivot mayoritarios
mayor.pivot(index=['id','fecha'], columns='param', values='v').reset_index()[0:5]

param,id,fecha,ca,cl,cond20,cond20situ,hco3,k,mg,na,no3,phsitu,so4
0,CA0708002,1981-10-06,96.199997,404.100006,1945.0,NaN,268.5,12.6,75.400002,267.200012,6.3,8.0,309.600006
1,CA0708002,1992-05-29,42.000000,468.000000,1981.0,NaN,278.0,NaN,72.000000,382.000000,11.0,6.9,325.000000
2,CA0708002,1992-09-10,67.000000,532.000000,3700.0,NaN,288.0,NaN,48.000000,513.000000,1.0,8.5,300.000000
3,CA0708002,1995-03-01,82.000000,384.000000,1848.0,NaN,257.0,10.0,47.000000,286.000000,2.0,7.8,231.000000
4,CA0708002,1995-09-13,60.000000,620.000000,2758.0,NaN,88.0,20.0,58.000000,382.000000,1.0,7.8,360.000000


In [14]:
# Como param cond20 y cond25 tienen casi siempe valores NaN 
# cargo otra ver mayor sin estos param
mayor = gwa[gwa['param'].isin(['na', 'k', 'ca', 'mg', 'cl', 'suo4', 'bicarb', 'co3', 'no3',
                             'cond20situ', 'phsitu'])]

In [15]:
# resultado final
mayor_pivot = mayor.pivot(index=['id','fecha'], columns='param', values='v').reset_index()
mayor_pivot[0:5]

param,id,fecha,bicarb,ca,cl,cond20situ,k,mg,na,no3,phsitu,suo4
0,CA0708002,1981-10-06,268.5,96.199997,404.100006,NaN,12.6,75.400002,267.200012,6.3,8.0,309.600006
1,CA0708002,1992-05-29,278.0,42.000000,468.000000,NaN,NaN,72.000000,382.000000,11.0,6.9,325.000000
2,CA0708002,1992-09-10,288.0,67.000000,532.000000,NaN,NaN,48.000000,513.000000,1.0,8.5,300.000000
3,CA0708002,1995-03-01,257.0,82.000000,384.000000,NaN,10.0,47.000000,286.000000,2.0,7.8,231.000000
4,CA0708002,1995-09-13,88.0,60.000000,620.000000,NaN,20.0,58.000000,382.000000,1.0,7.8,360.000000


In [16]:
# pongo los nombres de los param que necesito en el módulo AqueChemPlots
new_col_names1 = {'id':'Id', 'fecha':'Fecha',
                 'cl':'Cl', 'suo4':'SO4', 'bicarb':'HCO3', 'no3':'NO3',
                 'na':'Na', 'k':'K' ,'ca':'Ca', 'mg':'Mg',
                 'cond20situ': 'Cond20situ', 'phsitu': 'PH'}
mayor_pivot.rename(columns = new_col_names1, inplace = True)

# Para hacer algunos gráficos como el Piper necesito la columna CO3, si no está la pongo con valor 0
col_name = 'CO3' 
if col_name not in mayor_pivot.columns.tolist():
    mayor_pivot[col_name] = 0.

mayor_pivot[0:5]

param,Id,Fecha,HCO3,Ca,Cl,Cond20situ,K,Mg,Na,NO3,PH,SO4,CO3
0,CA0708002,1981-10-06,268.5,96.199997,404.100006,NaN,12.6,75.400002,267.200012,6.3,8.0,309.600006,0.0
1,CA0708002,1992-05-29,278.0,42.000000,468.000000,NaN,NaN,72.000000,382.000000,11.0,6.9,325.000000,0.0
2,CA0708002,1992-09-10,288.0,67.000000,532.000000,NaN,NaN,48.000000,513.000000,1.0,8.5,300.000000,0.0
3,CA0708002,1995-03-01,257.0,82.000000,384.000000,NaN,10.0,47.000000,286.000000,2.0,7.8,231.000000,0.0
4,CA0708002,1995-09-13,88.0,60.000000,620.000000,NaN,20.0,58.000000,382.000000,1.0,7.8,360.000000,0.0


## Error de balance de carga (cbe)

In [17]:
# Cálculo del CBE
cbe(mayor_pivot)

mayor_pivot[mayor_pivot['cbe'] > 5]


param,Id,Fecha,HCO3,Ca,Cl,Cond20situ,K,Mg,Na,NO3,...,rCO3,rHCO3,rNO3,rCa,rMg,rK,rNa,sum_anions,sum_cations,cbe
30,CA0708002,2019-05-30,235.0,81.0,439.0,104.0,9.8,48.0,272.0,3.0,...,0.0,3.851398,0.048383,4.042118,3.949805,0.250650,11.831350,16.761227,20.073923,8.993301
33,CA0708002,2020-04-22,235.0,73.0,332.0,1964.0,8.6,45.0,263.0,3.9,...,0.0,3.851398,0.062898,3.642896,3.702942,0.219958,11.439872,16.672428,19.005668,6.539701


In [18]:
# exporto mayor_pivots a fichero csv
fo = r'E:\LSGB\20231117_JCUNVRS\calidad_quim_sub\chs_sc_mayor.csv'
mayor_pivot.to_csv(fo, index=False)

## Creo fichero de datos para GWChemPLots
Primero creo una nueva dataframe llamada data_graph y luego la salvo en un fichero csv

* Si mis datos no tienen columna TDS la creo a partir de la suma de aniones y cationes
* Si mis datos tienen columna de NO3, se suma a la columna SO4 (si no deseas esto desmárcalo)
* Antes de salvar la dataframe a csv borro las filas con valores NaN
* También puedo descartar análisis con un valor de CBE determinado u otros filtros que me interesan
* Pero recuerda, siempre puedes hacer más manipulaciones editando fichero csv que has salvado

Formo data_graph

In [19]:
data_graph = pd.DataFrame()

data_graph['Sample'] = mayor_pivot['Id'] + '-' + mayor_pivot['Fecha'].dt.strftime('%Y-%m-%d')
data_graph['Label'] = mayor_pivot['Id']
data_graph['Color'] = 'b'
data_graph['Marker'] = 'o'
data_graph['Size'] = 30
data_graph['Alpha'] = 0.6
data_graph['PH'] = mayor_pivot['PH']
data_graph['Ca'] = mayor_pivot['Ca']
data_graph['Mg'] = mayor_pivot['Mg']
data_graph['Na'] = mayor_pivot['Na']
data_graph['K'] = mayor_pivot['K']
data_graph['HCO3'] = mayor_pivot['HCO3']
data_graph['CO3'] = mayor_pivot['CO3']
data_graph['Cl'] = mayor_pivot['Cl']
data_graph['SO4'] = mayor_pivot['SO4']
if 'NO3' in mayor_pivot:
    data_graph['SO4'] = data_graph['SO4'] + mayor_pivot['NO3']
if 'TDS' not in mayor_pivot:
    data_graph['TDS'] = mayor_pivot['sum_cations'] + np.abs(mayor_pivot['sum_anions'])
else:
    data_graph['TDS'] = mayor_pivot['TDS']
data_graph['CBE'] = mayor_pivot['cbe']

data_graph = data_graph.dropna()

data_graph[0:5]


,Sample,Label,Color,Marker,Size,Alpha,PH,Ca,Mg,Na,K,HCO3,CO3,Cl,SO4,TDS,CBE
0,CA0708002-1981-10-06,CA0708002,b,o,30,0.6,8.0,96.199997,75.400002,267.200012,12.6,268.5,0.0,404.100006,315.899994,45.295968,1.333300
3,CA0708002-1995-03-01,CA0708002,b,o,30,0.6,7.8,82.000000,47.000000,286.000000,10.0,257.0,0.0,384.000000,233.000000,40.540435,1.901321
4,CA0708002-1995-09-13,CA0708002,b,o,30,0.6,7.8,60.000000,58.000000,382.000000,20.0,88.0,0.0,620.000000,361.000000,51.335867,-3.013386
5,CA0708002-1996-06-05,CA0708002,b,o,30,0.6,7.9,72.000000,55.000000,289.000000,9.0,236.0,0.0,428.000000,233.000000,41.696846,0.342411
6,CA0708002-1996-11-03,CA0708002,b,o,30,0.6,8.7,39.000000,36.000000,270.000000,10.0,85.0,0.0,400.000000,250.000000,34.789220,-2.793576


Opcional: Aplico a data_graph otros filtros que me puedan interesar antes de salvarlo como csv

In [20]:
# Voy a borrar los análisis con un error de CBE maypr que un valor
error_limite = 5

print(f'Número de análisis eliminados por superar cbs {error_limite}:', len(data_graph[data_graph['CBE'] > error_limite]))

# Delete rows con cbe > error_limite (en relidad lo que hago es copiar los que están por debajo de error_limite)
data_graph = data_graph[data_graph['CBE'] <= error_limite]

Número de análisis eliminados por superar cbs 5: 2


Ahora salvo data_graph a csv

In [21]:
fo = r'E:\LSGB\20231117_JCUNVRS\calidad_quim_sub\chs_sc_AquaChemPlots_data.csv'
data_graph.to_csv(fo, index=False)
